In [1]:
import requests
import json
import datetime
import pandas as pd

### Important 

[Sign up here](https://www.alphavantage.co/support/#api-key) to get your API key and run the following code - it's free with a limit of 25 calls per day. 

In [3]:
# Replace with your Alpha Vantage API key
import api
API_KEY = api.key

### Building "Rolling Thunder" data
- retrieves 1 month of intraday data 
- at the 1min level 
- over normal and extended trading hours (4a-8p)

In [4]:
# Define function to download intraday data
# outputsize="compact" vs outputsize="full"
def get_intraday_data(ticker, month, interval="1min", outputsize="full"):
    """
    This function retrieves intraday data for a stock using Alpha Vantage API.

    Args:
      symbol (str): The ticker symbol of the stock.
      interval (str, optional): The data interval. Defaults to "1min".
      outputsize (str, optional): The data output size. Defaults to "compact".

    Returns:
      dict: A dictionary containing the retrieved intraday data (if successful), 
            or None if an error occurs.
    """

    # Base URL for the API call
    base_url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={ticker}&interval={interval}&month={month}&outputsize={outputsize}&apikey={API_KEY}"
    print("Click on this to see webpage version: ", base_url)
    # Make the API request
    response = requests.get(base_url)

    # Check for successful response
    if response.status_code == 200:
        return json.loads(response.text)
    else:
        print(f"Error: {response.status_code}")
        return None

In [5]:
# create dataframe to store data while looping through each company
cols = ['ticker', 'date', 'time', 'price']
df_intraday = pd.DataFrame(columns=cols) 

# confirm dataframe is empty on first go through
# will need to import once already being built rather than start from scratch
df_intraday

,ticker,date,time,price


# NOTE:

highly recommend changing the stock tickers below as we already have those listed

In [6]:
# Define ticker symbol and desired timeframe
tickers = ['AAPL', 'META', 'NVDA'] #can change to anything
months = ['2024-01', '2024-02', '2024-03', '2024-04']

In [10]:
start = datetime.datetime.now()

# Download intraday data
for ticker in tickers:
    for month in months:
        intraday_data = get_intraday_data(ticker, month)

        # Access the 'Time Series (1min)' section
        time_series = intraday_data['Time Series (1min)']
        
        new_row_list = []

        # Loop through each date-time key in the time series
        for date_time, prices in time_series.items():
            # Split the date-time string to get date and time separately
            date, time = date_time.split(" ")
            # Extract date, time, and open price
            open_price = prices['1. open']
            ## Print the formatted output
            # print(f"Ticker: {symbol}, Date: {date}, Time: {time}, Open Price: {open_price}")

            new_row_data = {'ticker': ticker,
                          'date': date, 
                          'time': time, 
                          'price': open_price}
            new_row_list.append(new_row_data)

        df_month = pd.DataFrame(new_row_list)  # Create DataFrame for the month's data
    
        # after running this, my guess is that pd.concat is actually an expensive operation
        # might make more sense to build a new dataframe for each ticker, then add to the bigger one
        df_intraday = pd.concat([df_intraday, df_month], ignore_index=True) 

Click on this to see webpage version:  https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=AAPL&interval=1min&month=2024-01&outputsize=full&apikey=1UUWAC2NCHRN36MF
Click on this to see webpage version:  https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=AAPL&interval=1min&month=2024-02&outputsize=full&apikey=1UUWAC2NCHRN36MF
Click on this to see webpage version:  https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=AAPL&interval=1min&month=2024-03&outputsize=full&apikey=1UUWAC2NCHRN36MF
Click on this to see webpage version:  https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=AAPL&interval=1min&month=2024-04&outputsize=full&apikey=1UUWAC2NCHRN36MF
Click on this to see webpage version:  https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=META&interval=1min&month=2024-01&outputsize=full&apikey=1UUWAC2NCHRN36MF
Click on this to see webpage version:  https://www.alphavantage.co/query?function=TIM

In [11]:
print(df_intraday.shape)
df_intraday.head()

(236388, 4)


,ticker,date,time,price
0,AAPL,2024-01-31,19:59:00,185.5340
1,AAPL,2024-01-31,19:58:00,185.3640
2,AAPL,2024-01-31,19:57:00,185.2840
3,AAPL,2024-01-31,19:56:00,185.2740
4,AAPL,2024-01-31,19:55:00,185.2540


In [12]:
filename = '-'.join(tickers)

df_intraday.to_csv(f'{filename}_intraday_data.csv')

### Building "Moving Avg-Mean Regression" data
- retrieves daily data (open, low, high close)
- for the past 20 years or the company's listing lifetime (whichever is shorter)


In [13]:
# Define function to download intraday data
# outputsize="compact" vs outputsize="full"
def get_open_close_data(ticker, outputsize="full"):
    """
    This function retrieves intraday data for a stock using Alpha Vantage API.

    Args:
      symbol (str): The ticker symbol of the stock.
      outputsize (str, optional): The data output size. Defaults to "compact".

    Returns:
      dict: A dictionary containing the retrieved intraday data (if successful), 
            or None if an error occurs.
    """

    # Base URL for the API call
    base_url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={ticker}&outputsize={outputsize}&apikey={API_KEY}"
    print("Click on this to see webpage version: ", base_url)
    # Make the API request
    response = requests.get(base_url)

    # Check for successful response
    if response.status_code == 200:
        return json.loads(response.text)
    else:
        print(f"Error: {response.status_code}")
        return None

In [14]:
# create dataframe to store data while looping through each company
cols = ['ticker', 'date', 'open', 'high', 'low', 'close']
df_daily = pd.DataFrame(columns=cols) 

# confirm dataframe is empty on first go through
# will need to import once already being built rather than start from scratch
df_daily

,ticker,date,open,high,low,close


# NOTE:

highly recommend changing the stock tickers below as we already have those listed

In [5]:
tickers = ['BA', 'F', 'TSLA'] #can change to anything

In [ ]:
for ticker in tickers:
    # Download intraday data
    daily_data = get_open_close_data(ticker)
    
    new_row_list = []
    
    # Access the 'Time Series (Daily)' section
    daily_data = daily_data['Time Series (Daily)']
    
    # Loop through each date in the daily data
    for date, prices in daily_data.items():
        # Extract open, high, low, and close prices
        open_price = prices['1. open']
        high_price = prices['2. high']
        low_price = prices['3. low']
        close_price = prices['4. close']

#         # Print all data on the same line, separated by commas
#         print(f"{date}, {open_price}, {high_price}, {low_price}, {close_price}")
        new_row_data = {'ticker': ticker,
                        'date': date, 
                        'open': open_price, 
                        'high': high_price, 
                        'low': low_price, 
                        'close': close_price}
    
        new_row_list.append(new_row_data)
    
    
    df_month = pd.DataFrame(new_row_list)# Create DataFrame for this tickers data
    
    df_daily = pd.concat([df_daily, df_month], ignore_index=True) 

In [ ]:
print(df_daily.shape)
df_daily.head()

In [ ]:
df_daily['ticker'].value_counts()

In [ ]:
filename = '-'.join(tickers)

df_daily.to_csv(f'{filename}_daily_data.csv')